# Parkway Project Use Case 1: Write Off Cases Prediction

## DATA PREPARATION

### Load Data

#### Import packages

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_rows', 500)
import warnings
warnings.filterwarnings(action='once')
import pickle
from imblearn.over_sampling import SMOTE

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
import os
os.chdir('../')
print(os.getcwd())

/home/jupyter/temp25Oct-Mok-cfc9083c3125ca7ce3bf62e5/PRS-PM-ISY5002-GROUP5/SystemCode


/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
%load_ext autoreload
%autoreload 2
from datapipeline import Datapipeline
dpl = Datapipeline()

#### Read data from file

In [10]:
file_dir = './Data Exploration/data/uc1/'
dict_X_train_file_paths = {
    'GHL' : [file_dir + 'GHL_new_train_X_uc1.pkl'],
#    'MEH' : [file_dir + 'MEH_new_train_X_uc1.pkl'],
#    'PEH' : [file_dir + 'PEH_new_train_X_uc1.pkl'],
#    'PNH' : [file_dir + 'PNH_new_train_X_uc1.pkl']
}
dict_y_train_file_paths = {
    'GHL' : file_dir + 'GHL_data_y_train.pkl',
#     'MEH' : file_dir + 'MEH_data_y_train.pkl',
#     'PEH' : file_dir + 'PEH_data_y_train.pkl',
#     'PNH' : file_dir + 'PNH_data_y_train.pkl'
}
dict_X_test_file_paths = {
    'GHL' : [file_dir + 'GHL_new_test_X_uc1.pkl'],
#     'MEH' : [file_dir + 'MEH_new_test_X_uc1.pkl'],
#     'PEH' : [file_dir + 'PEH_new_test_X_uc1.pkl'],
#     'PNH' : [file_dir + 'PNH_new_test_X_uc1.pkl']
}
dict_y_test_file_paths = {
    'GHL' : file_dir + 'GHL_data_y_test.pkl',
#     'MEH' : file_dir + 'MEH_data_y_test.pkl',
#     'PEH' : file_dir + 'PEH_data_y_test.pkl',
#     'PNH' : file_dir + 'PNH_data_y_test.pkl'
}

In [11]:
dict_df_X_train = {}
dict_df_y_train = {}
dict_df_X_test = {}
dict_df_y_test = {}

for hosp in dict_X_train_file_paths:
    dict_df_X_train[hosp] = pd.concat([pd.read_pickle(file_path)
                                       for file_path in dict_X_train_file_paths[hosp]])
    print(f'X_train {hosp} {dict_df_X_train[hosp].shape}')
    
    dict_df_y_train[hosp] = pd.read_pickle(dict_y_train_file_paths[hosp])
    print(f'y_train {hosp} {dict_df_y_train[hosp].shape}')

    dict_df_X_test[hosp] = pd.concat([pd.read_pickle(file_path)
                                      for file_path in dict_X_test_file_paths[hosp]])
    print(f'X_test {hosp} {dict_df_X_test[hosp].shape}')

    dict_df_y_test[hosp] = pd.read_pickle(dict_y_test_file_paths[hosp])
    print(f'y_test {hosp} {dict_df_y_test[hosp].shape}')

X_train GHL (79672, 3581)
y_train GHL (79672,)
X_test GHL (26557, 3581)
y_test GHL (26557,)


In [12]:
def convert_y(df):
    df1 = dpl.bin_column(df.to_frame(), col='WRITE_OFF', bin_thresh = [500])
    df1 = pd.get_dummies(df1['bin_WRITE_OFF'])
    df1 = df1.drop(0, axis=1)
    df1.columns = ['WRITE_OFF_LABEL']
    return df1

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
for hosp in dict_df_y_train:
    dict_df_y_train[hosp] = convert_y(dict_df_y_train[hosp])
    dict_df_y_test[hosp] = convert_y(dict_df_y_test[hosp])

## DECISION TREE

### Decision Tree WITHOUT SMOTE

In [14]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix  
from sklearn.tree import export_graphviz
from subprocess import call

In [15]:
criterion = 'entropy'
rand_seed = 0

In [16]:
for hosp in dict_df_X_train:
    model = DecisionTreeClassifier(criterion=criterion,random_state=rand_seed)
    model.fit(dict_df_X_train[hosp], dict_df_y_train[hosp])
    print(model)
    print(f"Accuracy on {hosp} training set without SMOTE decision tree: {model.score(dict_df_X_train[hosp], dict_df_y_train[hosp])}")
    print(f"Accuracy on {hosp} test set without SMOTE decision tree: {model.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")
    
    filename = file_dir + f'{hosp}_uc1_no_smote_dt_model.pkl'
    pickle.dump(model, open(filename, 'wb'))
    
    y_pred = model.predict(dict_df_X_test[hosp])
    print(hosp, 'decision tree\n', confusion_matrix(dict_df_y_test[hosp], y_pred))  
    print(hosp, 'decision tree\n', classification_report(dict_df_y_test[hosp], y_pred)) 

DecisionTreeClassifier(criterion='entropy', random_state=0)
Accuracy on GHL training set without SMOTE decision tree: 1.0
Accuracy on GHL test set without SMOTE decision tree: 0.9986444251986294


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/GHL_uc1_no_smote_dt_model.pkl'>
  if __name__ == '__main__':


GHL decision tree
 [[26520    18]
 [   18     1]]
GHL decision tree
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     26538
           1       0.05      0.05      0.05        19

    accuracy                           1.00     26557
   macro avg       0.53      0.53      0.53     26557
weighted avg       1.00      1.00      1.00     26557



### Decision Tree WITH SMOTE

#### Decision Tree WITH SMOTE SS = 0.1 ~ 1.0

In [17]:
for hosp in dict_df_X_train:
    for sample in np.arange(0.1, 1.1, 0.1):
        sm_ss = SMOTE(random_state=55,sampling_strategy=sample)
        X_train_ss, y_train_ss = sm_ss.fit_sample(dict_df_X_train[hosp], dict_df_y_train[hosp])
        
        model = DecisionTreeClassifier(criterion=criterion,random_state=rand_seed)
        model.fit(X_train_ss, y_train_ss)
        print(model)
        print(f"Accuracy on {hosp} training set with {sample} SMOTE decision tree: {model.score(X_train_ss, y_train_ss)}")
        print(f"Accuracy on {hosp} test set with {sample} SMOTE decision tree: {model.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")
    
        filename = file_dir + f'{hosp}_uc1_{sample}_smote_dt_model.pkl'
        pickle.dump(model, open(filename, 'wb'))
        
        y_pred = model.predict(dict_df_X_test[hosp])
        print(hosp, 'decision tree \n', confusion_matrix(dict_df_y_test[hosp], y_pred))  
        print(hosp, 'decision tree \n', classification_report(dict_df_y_test[hosp], y_pred)) 

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


DecisionTreeClassifier(criterion='entropy', random_state=0)
Accuracy on GHL training set with 0.1 SMOTE decision tree: 1.0
Accuracy on GHL test set with 0.1 SMOTE decision tree: 0.9986444251986294


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/GHL_uc1_0.1_smote_dt_model.pkl'>
  del sys.path[0]


GHL decision tree 
 [[26518    20]
 [   16     3]]
GHL decision tree 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     26538
           1       0.13      0.16      0.14        19

    accuracy                           1.00     26557
   macro avg       0.56      0.58      0.57     26557
weighted avg       1.00      1.00      1.00     26557

DecisionTreeClassifier(criterion='entropy', random_state=0)
Accuracy on GHL training set with 0.2 SMOTE decision tree: 1.0
Accuracy on GHL test set with 0.2 SMOTE decision tree: 0.9986444251986294


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/GHL_uc1_0.2_smote_dt_model.pkl'>
  del sys.path[0]


...

GHL decision tree 
 [[26516    22]
 [   17     2]]
GHL decision tree 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     26538
           1       0.08      0.11      0.09        19

    accuracy                           1.00     26557
   macro avg       0.54      0.55      0.55     26557
weighted avg       1.00      1.00      1.00     26557

DecisionTreeClassifier(criterion='entropy', random_state=0)
Accuracy on GHL training set with 0.9 SMOTE decision tree: 1.0
Accuracy on GHL test set with 0.9 SMOTE decision tree: 0.9984184960650676


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/GHL_uc1_0.9_smote_dt_model.pkl'>
  del sys.path[0]


GHL decision tree 
 [[26514    24]
 [   18     1]]
GHL decision tree 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     26538
           1       0.04      0.05      0.05        19

    accuracy                           1.00     26557
   macro avg       0.52      0.53      0.52     26557
weighted avg       1.00      1.00      1.00     26557

DecisionTreeClassifier(criterion='entropy', random_state=0)
Accuracy on GHL training set with 1.0 SMOTE decision tree: 1.0
Accuracy on GHL test set with 1.0 SMOTE decision tree: 0.9982678766426931


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/GHL_uc1_1.0_smote_dt_model.pkl'>
  del sys.path[0]


GHL decision tree 
 [[26510    28]
 [   18     1]]
GHL decision tree 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     26538
           1       0.03      0.05      0.04        19

    accuracy                           1.00     26557
   macro avg       0.52      0.53      0.52     26557
weighted avg       1.00      1.00      1.00     26557



## LOGISTIC REGRESSION

### Logistic Regression Without SMOTE

In [18]:
from sklearn.linear_model import LogisticRegression

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [19]:
reg_strength = 0.01

In [20]:
for hosp in dict_df_X_train:
    model = LogisticRegression(C=reg_strength).fit(dict_df_X_train[hosp], dict_df_y_train[hosp])
    print(model)
    print(f"{hosp} Training set without SMOTE log reg score: {model.score(dict_df_X_train[hosp], dict_df_y_train[hosp])}")
    print(f"{hosp} Test set without SMOTE log reg score: {model.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")
    
    filename = file_dir + f'{hosp}_uc1_no_smote_logreg_model.pkl'
    pickle.dump(model, open(filename, 'wb'))
    
    y_pred = model.predict(dict_df_X_test[hosp])
    print(hosp, 'log reg \n', confusion_matrix(dict_df_y_test[hosp], y_pred))  
    print(hosp, 'log reg \n', classification_report(dict_df_y_test[hosp], y_pred)) 

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


LogisticRegression(C=0.01)
GHL Training set without SMOTE log reg score: 0.9990460889647556
GHL Test set without SMOTE log reg score: 0.999284557743721


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/GHL_uc1_no_smote_logreg_model.pkl'>
  


GHL log reg 
 [[26538     0]
 [   19     0]]
GHL log reg 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     26538
           1       0.00      0.00      0.00        19

    accuracy                           1.00     26557
   macro avg       0.50      0.50      0.50     26557
weighted avg       1.00      1.00      1.00     26557



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Logistic Regression With SMOTE

#### Logistic Regression With SMOTE ss=0.1 ~ 1.0

In [21]:
for hosp in dict_df_X_train:
    for sample in np.arange(0.1, 1.1, 0.1):
        sm_ss = SMOTE(random_state=55,sampling_strategy=sample)
        X_train_ss, y_train_ss = sm_ss.fit_sample(dict_df_X_train[hosp], dict_df_y_train[hosp])
        
        model = LogisticRegression(C=reg_strength).fit(X_train_ss, y_train_ss)
        print(model)
        print(f"{hosp} Training set with {sample} SMOTE log reg score: {model.score(X_train_ss, y_train_ss)}")
        print(f"{hosp} Test set with {sample} SMOTE log reg score: {model.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")

        filename = file_dir + f'{hosp}_uc1_{sample}_smote_logreg_model.pkl'
        pickle.dump(model, open(filename, 'wb'))

        y_pred = model.predict(dict_df_X_test[hosp])
        print(hosp, 'log reg \n', confusion_matrix(dict_df_y_test[hosp], y_pred))  
        print(hosp, 'log reg \n', classification_report(dict_df_y_test[hosp], y_pred)) 

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    htt

LogisticRegression(C=0.01)
GHL Training set with 0.1 SMOTE log reg score: 0.9622869934670383
GHL Test set with 0.1 SMOTE log reg score: 0.9956320367511391


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/GHL_uc1_0.1_smote_logreg_model.pkl'>
  if sys.path[0] == '':


GHL log reg 
 [[26438   100]
 [   16     3]]
GHL log reg 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     26538
           1       0.03      0.16      0.05        19

    accuracy                           1.00     26557
   macro avg       0.51      0.58      0.52     26557
weighted avg       1.00      1.00      1.00     26557



/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


...

GHL log reg 
 [[25793   745]
 [   10     9]]
GHL log reg 
               precision    recall  f1-score   support

           0       1.00      0.97      0.99     26538
           1       0.01      0.47      0.02        19

    accuracy                           0.97     26557
   macro avg       0.51      0.72      0.50     26557
weighted avg       1.00      0.97      0.98     26557



/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=0.01)
GHL Training set with 1.0 SMOTE log reg score: 0.9844654949307134
GHL Test set with 1.0 SMOTE log reg score: 0.9696125315359415


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/GHL_uc1_1.0_smote_logreg_model.pkl'>
  if sys.path[0] == '':


GHL log reg 
 [[25741   797]
 [   10     9]]
GHL log reg 
               precision    recall  f1-score   support

           0       1.00      0.97      0.98     26538
           1       0.01      0.47      0.02        19

    accuracy                           0.97     26557
   macro avg       0.51      0.72      0.50     26557
weighted avg       1.00      0.97      0.98     26557



## NAIVE BAYES

### Naive Bayes Without SMOTE

In [22]:
from sklearn.naive_bayes import GaussianNB

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
for hosp in dict_df_X_train:
    # Initiating the Gaussian Classifier
    model = GaussianNB()

    # Training your model 
    model.fit(dict_df_X_train[hosp], dict_df_y_train[hosp])
    print(model)
    filename = file_dir + f'{hosp}_uc1_no_smote_nb_model.pkl'
    pickle.dump(model, open(filename, 'wb'))

    # Score
    print(f"{hosp} Training set without SMOTE Naive Bayes score: {model.score(dict_df_X_train[hosp], dict_df_y_train[hosp])}")
    print(f"{hosp} Test set without SMOTE Naive Bayes score: {model.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")

    # Confusion Matrix
    y_pred = model.predict(dict_df_X_test[hosp])
    print(hosp, 'Naive Bayes \n', confusion_matrix(dict_df_y_test[hosp], y_pred))  
    print(hosp, 'Naive Bayes \n', classification_report(dict_df_y_test[hosp], y_pred)) 

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


GaussianNB()


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/GHL_uc1_no_smote_nb_model.pkl'>
  if __name__ == '__main__':


GHL Training set without SMOTE Naive Bayes score: 0.9772065468420524
GHL Test set without SMOTE Naive Bayes score: 0.9765033701095757
GHL Naive Bayes 
 [[25929   609]
 [   15     4]]
GHL Naive Bayes 
               precision    recall  f1-score   support

           0       1.00      0.98      0.99     26538
           1       0.01      0.21      0.01        19

    accuracy                           0.98     26557
   macro avg       0.50      0.59      0.50     26557
weighted avg       1.00      0.98      0.99     26557



### Naive Bayes With SMOTE

#### Naive Bayes Without SMOTE ss=0.1 ~ 1.0

In [24]:
for hosp in dict_df_X_train:
    for sample in np.arange(0.1, 1.1, 0.1):
        sm_ss = SMOTE(random_state=55,sampling_strategy=sample)
        X_train_ss, y_train_ss = sm_ss.fit_sample(dict_df_X_train[hosp], dict_df_y_train[hosp])
        
        # Initiating the Gaussian Classifier
        model = GaussianNB()
        
        # Training your model 
        model.fit(X_train_ss, y_train_ss)
        print(model)
        filename = file_dir + f'{hosp}_uc1_{sample}_smote_nb_model.pkl'
        pickle.dump(model, open(filename, 'wb'))

        # Score
        print(f"{hosp} Training set with {sample} SMOTE Naive Bayes score: {model.score(X_train_ss, y_train_ss)}")
        print(f"{hosp} Test set with {sample} SMOTE Naive Bayes score: {model.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")

        # Confusion Matrix
        y_pred = model.predict(dict_df_X_test[hosp])
        print(hosp, 'Naive Bayes\n', confusion_matrix(dict_df_y_test[hosp], y_pred))  
        print(hosp, 'Naive Bayes\n', classification_report(dict_df_y_test[hosp], y_pred)) 

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


GaussianNB()


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/GHL_uc1_0.1_smote_nb_model.pkl'>
  del sys.path[0]


GHL Training set with 0.1 SMOTE Naive Bayes score: 0.9792589885330532
GHL Test set with 0.1 SMOTE Naive Bayes score: 0.9765033701095757
GHL Naive Bayes
 [[25929   609]
 [   15     4]]
GHL Naive Bayes
               precision    recall  f1-score   support

           0       1.00      0.98      0.99     26538
           1       0.01      0.21      0.01        19

    accuracy                           0.98     26557
   macro avg       0.50      0.59      0.50     26557
weighted avg       1.00      0.98      0.99     26557



/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


...

GHL Training set with 0.9 SMOTE Naive Bayes score: 0.9879921181744846
GHL Test set with 0.9 SMOTE Naive Bayes score: 0.9765033701095757
GHL Naive Bayes
 [[25929   609]
 [   15     4]]
GHL Naive Bayes
               precision    recall  f1-score   support

           0       1.00      0.98      0.99     26538
           1       0.01      0.21      0.01        19

    accuracy                           0.98     26557
   macro avg       0.50      0.59      0.50     26557
weighted avg       1.00      0.98      0.99     26557



/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


GaussianNB()


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/GHL_uc1_1.0_smote_nb_model.pkl'>
  del sys.path[0]


GHL Training set with 1.0 SMOTE Naive Bayes score: 0.9885925348945312
GHL Test set with 1.0 SMOTE Naive Bayes score: 0.9765033701095757
GHL Naive Bayes
 [[25929   609]
 [   15     4]]
GHL Naive Bayes
               precision    recall  f1-score   support

           0       1.00      0.98      0.99     26538
           1       0.01      0.21      0.01        19

    accuracy                           0.98     26557
   macro avg       0.50      0.59      0.50     26557
weighted avg       1.00      0.98      0.99     26557



## NEURAL NET MLP

### Neural Net MLP Without SMOTE, 2 Layers

In [25]:
from sklearn.neural_network import MLPClassifier  
from sklearn import metrics

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [26]:
list_hid_layers = [(128,128), (16,16,16), (32,32,32), (16,16,16,16), (32,32,32,32)]
max_iter = 1000
list_smote_sampling = [0.1, 0.2, 0.3, 0.5, 1.0]

#### Neural Net MLPClassifier without SMOTE

In [28]:
for hosp in dict_df_X_train:
    for hid_layers in list_hid_layers:
        model = MLPClassifier(hidden_layer_sizes=hid_layers, max_iter=max_iter,verbose=1)  
        model.fit(dict_df_X_train[hosp], dict_df_y_train[hosp])
        print(model)
        filename = file_dir + f'{hosp}_uc1_no_smote_mlp_{hid_layers}_model.pkl'
        pickle.dump(model, open(filename, 'wb'))

        predictions = model.predict(dict_df_X_test[hosp]) 

        # Score
        print(f"{hosp} Training set without SMOTE mlp {hid_layers} score: {model.score(dict_df_X_train[hosp], dict_df_y_train[hosp])}")
        print(f"{hosp} Test set without SMOTE mlp {hid_layers} score: {model.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")

        print(f"{hosp} Accuracy without SMOTE mlp {hid_layers}: ", metrics.accuracy_score(dict_df_y_test[hosp], predictions))
        print(hosp, f'mlp {hid_layers}\n', confusion_matrix(dict_df_y_test[hosp],predictions))  
        print(hosp, f'mlp {hid_layers}\n', classification_report(dict_df_y_test[hosp],predictions))  

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Iteration 1, loss = 0.02600972
Iteration 2, loss = 0.00579728
Iteration 3, loss = 0.00449412
Iteration 4, loss = 0.00345465
Iteration 5, loss = 0.00242810
Iteration 6, loss = 0.00177766
Iteration 7, loss = 0.00144586
Iteration 8, loss = 0.00087191
Iteration 9, loss = 0.00068203
Iteration 10, loss = 0.00045496
Iteration 11, loss = 0.00049959
Iteration 12, loss = 0.00029035
Iteration 13, loss = 0.00053392
Iteration 14, loss = 0.00059042
Iteration 15, loss = 0.00056622
Iteration 16, loss = 0.00047071
Iteration 17, loss = 0.00068893
Iteration 18, loss = 0.00085757
Iteration 19, loss = 0.00042698
Iteration 20, loss = 0.00019457
Iteration 21, loss = 0.00016587
Iteration 22, loss = 0.00020288
Iteration 23, loss = 0.00012714
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
MLPClassifier(hidden_layer_sizes=(128, 128), max_iter=1000, verbose=1)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/GHL_uc1_no_smote_mlp_(128, 128)_model.pkl'>
  import sys


GHL Training set without SMOTE mlp (128, 128) score: 0.9999874485390099
GHL Test set without SMOTE mlp (128, 128) score: 0.9990209737545657
GHL Accuracy without SMOTE mlp (128, 128):  0.9990209737545657
GHL mlp (128, 128)
 [[26530     8]
 [   18     1]]
GHL mlp (128, 128)
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     26538
           1       0.11      0.05      0.07        19

    accuracy                           1.00     26557
   macro avg       0.56      0.53      0.54     26557
weighted avg       1.00      1.00      1.00     26557



/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


...

GHL Training set without SMOTE mlp (16, 16, 16, 16) score: 1.0
GHL Test set without SMOTE mlp (16, 16, 16, 16) score: 0.9990962834657529
GHL Accuracy without SMOTE mlp (16, 16, 16, 16):  0.9990962834657529
GHL mlp (16, 16, 16, 16)
 [[26531     7]
 [   17     2]]
GHL mlp (16, 16, 16, 16)
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     26538
           1       0.22      0.11      0.14        19

    accuracy                           1.00     26557
   macro avg       0.61      0.55      0.57     26557
weighted avg       1.00      1.00      1.00     26557



/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Iteration 1, loss = 0.03175654
Iteration 2, loss = 0.00602640
Iteration 3, loss = 0.00495211
Iteration 4, loss = 0.00385336
Iteration 5, loss = 0.00323167
Iteration 6, loss = 0.00208605
Iteration 7, loss = 0.00133545
Iteration 8, loss = 0.00130436
Iteration 9, loss = 0.00074133
Iteration 10, loss = 0.00047921
Iteration 11, loss = 0.00055340
Iteration 12, loss = 0.00037132
Iteration 13, loss = 0.00031490
Iteration 14, loss = 0.00064364
Iteration 15, loss = 0.00023140
Iteration 16, loss = 0.00032342
Iteration 17, loss = 0.00121134
Iteration 18, loss = 0.00060589
Iteration 19, loss = 0.00015037
Iteration 20, loss = 0.00036148
Iteration 21, loss = 0.00072258
Iteration 22, loss = 0.00021675
Iteration 23, loss = 0.00020858
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
MLPClassifier(hidden_layer_sizes=(32, 32, 32, 32), max_iter=1000, verbose=1)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/GHL_uc1_no_smote_mlp_(32, 32, 32, 32)_model.pkl'>
  import sys


GHL Training set without SMOTE mlp (32, 32, 32, 32) score: 0.9998870368510895
GHL Test set without SMOTE mlp (32, 32, 32, 32) score: 0.9991715931769402
GHL Accuracy without SMOTE mlp (32, 32, 32, 32):  0.9991715931769402
GHL mlp (32, 32, 32, 32)
 [[26535     3]
 [   19     0]]
GHL mlp (32, 32, 32, 32)
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     26538
           1       0.00      0.00      0.00        19

    accuracy                           1.00     26557
   macro avg       0.50      0.50      0.50     26557
weighted avg       1.00      1.00      1.00     26557



### Neural Net With SMOTE

In [29]:
list_hid_layers = [(16,16), (32,32), (64,64), (128,128), (256,256),
                   (16,16,16), (32,32,32), (64,64,64),
                   (16,16,16,16), (32,32,32,32)]
max_iter = 1000
list_smote_sampling = [0.1, 0.2, 0.3, 0.5, 1.0]

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### Neural Net Layers With SMOTE

In [ ]:
for hosp in dict_df_X_train:
    for hid_layers in list_hid_layers:
        for sample in list_smote_sampling:
            sm_ss = SMOTE(random_state=55,sampling_strategy=sample)
            X_train_ss, y_train_ss = sm_ss.fit_sample(dict_df_X_train[hosp], dict_df_y_train[hosp])
        
            model = MLPClassifier(hidden_layer_sizes=hid_layers, max_iter=max_iter,verbose=1)  
            model.fit(X_train_ss, y_train_ss)  
            print(model)
            filename = file_dir + f'{hosp}_uc1_{sample}_smote_mlp_{hid_layers}_model.pkl'
            pickle.dump(model, open(filename, 'wb'))
            
            predictions = model.predict(dict_df_X_test[hosp]) 

            # Score
            print(f"{hosp} Training set {sample} SMOTE mlp {hid_layers} score: {model.score(X_train_ss, y_train_ss)}")
            print(f"{hosp} Test set {sample} SMOTE mlp {hid_layers} score: {model.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")

            print(f"{hosp} Accuracy: ", metrics.accuracy_score(dict_df_y_test[hosp], predictions))
            print(hosp, f'mlp {hid_layers}\n', confusion_matrix(dict_df_y_test[hosp],predictions))  
            print(hosp, f'mlp {hid_layers}\n', classification_report(dict_df_y_test[hosp],predictions))  

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Iteration 1, loss = 0.20007886
Iteration 2, loss = 0.00851135
Iteration 3, loss = 0.00335019
Iteration 4, loss = 0.00185529
Iteration 5, loss = 0.00213162
Iteration 6, loss = 0.00085333
Iteration 7, loss = 0.00056258
Iteration 8, loss = 0.00044879
Iteration 9, loss = 0.00035732
Iteration 10, loss = 0.00028566
Iteration 11, loss = 0.00025900
Iteration 12, loss = 0.00025362
Iteration 13, loss = 0.00022349
Iteration 14, loss = 0.00100574
Iteration 15, loss = 0.00020332
Iteration 16, loss = 0.00017062
Iteration 17, loss = 0.00014926
Iteration 18, loss = 0.00015908
Iteration 19, loss = 0.00012338
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
MLPClassifier(hidden_layer_sizes=(16, 16), max_iter=1000, verbose=1)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/GHL_uc1_0.1_smote_mlp_(16, 16)_model.pkl'>
  # This is added back by InteractiveShellApp.init_path()


GHL Training set 0.1 SMOTE mlp (16, 16) score: 1.0
GHL Test set 0.1 SMOTE mlp (16, 16) score: 0.9988703543321912
GHL Accuracy:  0.9988703543321912
GHL mlp (16, 16)
 [[26525    13]
 [   17     2]]
GHL mlp (16, 16)
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     26538
           1       0.13      0.11      0.12        19

    accuracy                           1.00     26557
   macro avg       0.57      0.55      0.56     26557
weighted avg       1.00      1.00      1.00     26557



/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


...

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/GHL_uc1_0.3_smote_mlp_(16, 16, 16)_model.pkl'>
  # This is added back by InteractiveShellApp.init_path()


GHL Training set 0.3 SMOTE mlp (16, 16, 16) score: 0.9999806718466118
GHL Test set 0.3 SMOTE mlp (16, 16, 16) score: 0.9988703543321912
GHL Accuracy:  0.9988703543321912
GHL mlp (16, 16, 16)
 [[26525    13]
 [   17     2]]
GHL mlp (16, 16, 16)
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     26538
           1       0.13      0.11      0.12        19

    accuracy                           1.00     26557
   macro avg       0.57      0.55      0.56     26557
weighted avg       1.00      1.00      1.00     26557



/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Iteration 1, loss = 0.09758943
Iteration 2, loss = 0.00328239
Iteration 3, loss = 0.00140405
Iteration 4, loss = 0.00171498
Iteration 5, loss = 0.00053133
Iteration 6, loss = 0.00033631
Iteration 7, loss = 0.00027707
Iteration 8, loss = 0.00022196
Iteration 9, loss = 0.00085604
Iteration 10, loss = 0.00028551
Iteration 11, loss = 0.00025630
Iteration 12, loss = 0.00027096
Iteration 13, loss = 0.00010701
Iteration 14, loss = 0.00157055
Iteration 15, loss = 0.00024787
Iteration 16, loss = 0.00010703
Iteration 17, loss = 0.00012660
Iteration 18, loss = 0.00008074
Iteration 19, loss = 0.00011376
Iteration 20, loss = 0.00006681
Iteration 21, loss = 0.00008394
Iteration 22, loss = 0.00085089
Iteration 23, loss = 0.00009436
Iteration 24, loss = 0.00007545
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
MLPClassifier(hidden_layer_sizes=(16, 16, 16), max_iter=1000, verbose=1)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/GHL_uc1_0.5_smote_mlp_(16, 16, 16)_model.pkl'>
  # This is added back by InteractiveShellApp.init_path()


## KNN

In [ ]:
# Be careful when running this, about 2 full days are needed to run this!!!
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
for hosp in dict_df_X_train:
    test_scores = []
    train_scores = []

    for i in range(1,15):

        knn = KNeighborsClassifier(i)
        knn.fit(dict_df_X_train[hosp],dict_df_y_train[hosp])

        train_scores.append(knn.score(dict_df_X_train[hosp],dict_df_y_train[hosp]))
        test_scores.append(knn.score(dict_df_X_test[hosp],dict_df_y_test[hosp]))

    ## score that comes from testing on the same datapoints that were used for training
    max_train_score = max(train_scores)
    train_scores_ind = [i for i, v in enumerate(train_scores) if v == max_train_score]
    print(f'{hosp} Max train score {max_train_score*100} % and k = {list(map(lambda x: x+1, train_scores_ind))}')

    ## score that comes from testing on the datapoints that were split in the beginning to be used for testing solely
    max_test_score = max(test_scores)
    test_scores_ind = [i for i, v in enumerate(test_scores) if v == max_test_score]
    print(f'{hosp} Max test score {max_test_score*100} % and k = {list(map(lambda x: x+1, test_scores_ind))}')

    plt.figure(figsize=(12,5))
    plt.plot(range(1,15),train_scores,marker='*',label='Train Score')
    plt.plot(range(1,15),test_scores,marker='o',label='Test Score')
    plt.title(f'{hosp} KNN')
    plt.tight_layout()
    plt.show()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
# From the above, the best KNN is at k=8
#Setup a knn classifier with k neighbors
best_knn = KNeighborsClassifier(8)

### KNN without SMOTE

In [ ]:
n_neighbours = 8
for hosp in dict_df_X_train:
    # Fit the KNN model with training data and score the test data
    best_knn = KNeighborsClassifier(n_neighbours)
    
    best_knn.fit(dict_df_X_train[hosp],dict_df_y_train[hosp])
    best_knn.score(dict_df_X_test[hosp],dict_df_y_test[hosp])
    
    # save the model to disk
    filename = f'{hosp}_best_knn_without_smote.pkl'
    pickle.dump(best_knn, open(filename, 'wb'))
    
    y_pred = best_knn.predict(dict_df_X_test[hosp])
    # Score for Best KNN, No SMOTE
    print(f"{hosp} Training set without SMOTE KNN({n_neighbours}) score: {best_knn.score(dict_df_X_train[hosp], dict_df_y_train[hosp])}")
    print(f"{hosp} Test set without SMOTE KNN({n_neighbours}) score: {best_knn.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")

    print(f"{hosp} Accuracy without SMOTE KNN({n_neighbours}): {metrics.accuracy_score(dict_df_y_test[hosp], y_pred)}")
    print(hosp, f'KNN({n_neighbours})\n', confusion_matrix(dict_df_y_test[hosp],y_pred))  
    print(hosp, f'KNN({n_neighbours})\n', classification_report(dict_df_y_test[hosp],y_pred))  

### KNN With SMOTE

#### KNN With SMOTE ss=0.1

In [ ]:
n_neighbours = 8
list_smote_sampling = [0.1, 0.5, 1.0]
for hosp in dict_df_X_train:
    for sample in list_smote_sampling:
        sm_ss = SMOTE(random_state=55,sampling_strategy=sample)
        X_train_ss, y_train_ss = sm_ss.fit_sample(dict_df_X_train[hosp], dict_df_y_train[hosp])
        
        # Fit the KNN model with training data and score the test data
        best_knn = KNeighborsClassifier(n_neighbours)    
        best_knn.fit(X_train_ss,y_train_ss)
        #best_knn.score(dict_df_X_test[hosp],dict_df_y_test[hosp])
        
        # save the model to disk
        filename = f'{hosp}_best_knn_{sample}_smote.pkl'
        pickle.dump(best_knn, open(filename, 'wb'))
        
        y_pred = best_knn.predict(dict_df_X_test[hosp])
        # Score for Best KNN, with SMOTE
        print(f"{hosp} Training set {sample} SMOTE KNN({n_neighbours}) score: {best_knn.score(X_train_ss, y_train_ss)}")
        print(f"{hosp} Test set {sample} SMOTE KNN({n_neighbours}) score: {best_knn.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")

        print(f"{hosp} Accuracy {sample} SMOTE KNN({n_neighbours}): ", metrics.accuracy_score(dict_df_y_test[hosp], y_pred))
        print(confusion_matrix(dict_df_y_test[hosp],y_pred))  
        print(classification_report(dict_df_y_test[hosp],y_pred)) 